In [10]:
import pandas as pd
import time

past = pd.read_csv("./past.csv", usecols=["Sex", "AGE", "Region", "TIME", "Value"])
past.columns = ["sex", "age_cat", "name", "year", "value"]
past.sex = past.sex.apply(lambda x: x.lower()[0])

def fix_year(year):
    return int(year[:-3]) if "(" in year else int(year)

proj_f = pd.read_csv("./projections_females.csv", thousands=",")
proj_f.Year = proj_f.Year.apply(fix_year)

proj_m = pd.read_csv("./projections_males.csv", thousands=",")
proj_m.Year = proj_m.Year.apply(fix_year)

proj_p = pd.read_csv("./projections_people.csv", thousands=",")
proj_p.Year = proj_p.Year.apply(fix_year)

age_cats = []
for x in range(0, 90, 5):
    label = f"{x}-{x + 4} years" if x < 85 else "85+ years"
    upper = (x + 4 if x < 85 else "-")
    age_cats.append([
        label,
        x,
        upper
    ])
    
data = pd.DataFrame(columns=["year", "district", "sex"] + [x[0] for x in age_cats])

years = [x for x in range(2001, 2059)]
sexes = ["m", "f", "p"]

start = time.time()
for year in years:
    for sex in sexes:
        for district in past.name.unique():
            row_data = [year, district, sex]
            for i, age_cat in enumerate(age_cats):
                if year < 2020:
                    row_data.append(past[(past.age_cat == list(past.age_cat.unique())[i]) & (past.sex == sex) & (past.name == district) & (past.year == year)].iat[0, 4])
                else:
                    data_this = proj_m if sex == "m" else proj_f if sex == "f" else proj_p                    
                    if age_cat[1] > 80:
                        age_bucket = data_this[(data_this.Year == year) & (data_this.District == district)].iat[0, 87]
                    else:
                        age_bucket = 0                        
                        for age in range(age_cat[1], age_cat[2] + 1):
                            age_bucket = age_bucket + data_this[(data_this.Year == year) & (data_this.District == district)].iat[0, age + 2]
                    row_data.append(age_bucket)
            data.loc[len(data)] = row_data
data

SyntaxError: invalid syntax (<ipython-input-10-6adb16054911>, line 43)

In [8]:
proj_f

,Year,District,0,1,2,3,4,5,6,7,...,77,78,79,80,81,82,83,84,85+,FEMALES
0,2017,Belconnen,628,666,673,679,652,646,605,650,...,223,208,205,200,174,162,160,122,844,49603
1,2017,Canberra East,6,2,16,8,2,8,0,4,...,5,5,1,0,0,1,1,3,10,479
2,2017,Gungahlin,686,634,742,655,701,640,624,606,...,94,66,70,65,58,38,28,32,108,38150
3,2017,North Canberra,251,278,238,215,235,219,234,231,...,100,116,122,115,111,111,93,105,672,27959
4,2017,South Canberra,133,110,136,141,129,164,131,144,...,87,79,73,61,76,59,55,81,564,14781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,2058,Weston Creek,170,168,173,171,163,174,169,170,...,125,117,113,102,95,97,92,91,622,13075
458,2058,Woden,274,271,275,264,287,264,289,294,...,242,233,217,208,210,194,195,185,1230,28030
459,2058,Molonglo,351,356,389,375,370,334,365,363,...,237,214,208,188,192,176,166,165,1034,31011
460,2058,Urriarra-Namadgi,6,5,4,4,4,5,4,4,...,1,1,1,2,3,1,2,2,10,278


In [9]:
age_cats

[['0-4 years', 0, 4],
 ['5-9 years', 5, 9],
 ['10-14 years', 10, 14],
 ['15-19 years', 15, 19],
 ['20-24 years', 20, 24],
 ['25-29 years', 25, 29],
 ['30-34 years', 30, 34],
 ['35-39 years', 35, 39],
 ['40-44 years', 40, 44],
 ['45-49 years', 45, 49],
 ['50-54 years', 50, 54],
 ['55-59 years', 55, 59],
 ['60-64 years', 60, 64],
 ['65-69 years', 65, 69],
 ['70-74 years', 70, 74],
 ['75-79 years', 75, 79],
 ['80-84 years', 80, 84],
 ['85+ years', 85, '-']]

In [34]:
sex = "m"
data_this = proj_m if sex == "m" else proj_f if sex == "f" else proj_p
data_this[(data_this.Year == 2017) & (data_this.District == "Belconnen")].iat[0, 87]

485